<a href="https://colab.research.google.com/github/AhmadAlabrash/NLP/blob/main/NLP_gensim_%26_spacy_%26_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


df = pd.read_csv("/content/Fake_Real_Data.csv")

In [2]:
df['label_num'] = df['label'].map({'Fake' : 0, 'Real': 1})

In [3]:
df

,Text,label,label_num
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0
1,U.S. conservative leader optimistic of common ...,Real,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0
4,Democrats say Trump agrees to work on immigrat...,Real,1
...,...,...,...
9895,Wikileaks Admits To Screwing Up IMMENSELY Wit...,Fake,0
9896,Trump consults Republican senators on Fed chie...,Real,1
9897,Trump lawyers say judge lacks jurisdiction for...,Real,1
9898,WATCH: Right-Wing Pastor Falsely Credits Trum...,Fake,0


In [5]:
import spacy
import spacy.cli
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [12]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[=================================================-] 98.3% 1634.1/1662.8MB downloaded


In [33]:
wv.get_vector('word')

array([ 3.59375000e-01,  4.15039062e-02,  9.03320312e-02,  5.46875000e-02,
       -1.47460938e-01,  4.76074219e-02, -8.49609375e-02, -2.04101562e-01,
        3.10546875e-01, -1.05590820e-02, -6.15234375e-02, -1.55273438e-01,
       -1.52343750e-01,  8.54492188e-02, -2.70996094e-02,  3.84765625e-01,
        4.78515625e-02,  2.58789062e-02,  4.49218750e-02, -2.79296875e-01,
        9.09423828e-03,  4.08203125e-01,  2.40234375e-01, -3.06640625e-01,
       -1.80664062e-01,  4.73632812e-02, -2.63671875e-01,  9.08203125e-02,
        1.37695312e-01, -7.20977783e-04,  2.67333984e-02,  1.92382812e-01,
       -2.29492188e-02,  9.70458984e-03, -7.37304688e-02,  4.29687500e-01,
       -7.93457031e-03,  1.06445312e-01,  2.80761719e-02, -2.29492188e-01,
       -1.91650391e-02, -2.36816406e-02,  3.51562500e-02,  1.71875000e-01,
       -1.12304688e-01,  6.25000000e-02, -1.69921875e-01,  1.29882812e-01,
       -1.54296875e-01,  1.58203125e-01, -7.76367188e-02,  1.78710938e-01,
       -1.72851562e-01,  

In [ ]:
for i in wv.vocab :
   print(i)

In [58]:
def preprocess(txt):
 after_process =[]
 for word in nlp(txt):
   
   if word not in wv.vocab or word.is_stop or word.is_punct or word.text == ' ' or word.is_alpha == False:
     continue
   after_process.append(wv.get_vector(word.lemma_) ) 
 
 return    after_process 

In [59]:
preprocess("Don't worry if you don't understand")

[]

In [ ]:
df['vector'] = df['Text'].apply(lambda text: preprocess(text))

In [54]:
df

,Text,label,label_num,vector
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0,[]
1,U.S. conservative leader optimistic of common ...,Real,1,[]
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1,[]
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0,[]
4,Democrats say Trump agrees to work on immigrat...,Real,1,[]
...,...,...,...,...
9895,Wikileaks Admits To Screwing Up IMMENSELY Wit...,Fake,0,[]
9896,Trump consults Republican senators on Fed chie...,Real,1,[]
9897,Trump lawyers say judge lacks jurisdiction for...,Real,1,[]
9898,WATCH: Right-Wing Pastor Falsely Credits Trum...,Fake,0,[]


In [48]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    df.vector.values, 
    df.label_num, 
    test_size=0.2, 
)

In [50]:
import numpy as np

X_train_2d = np.stack(x_train)
X_test_2d =  np.stack(x_test)

In [53]:
X_train_2d

array([], shape=(7920, 0), dtype=float64)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

# creating a GradientBoosting model object
clf = GradientBoostingClassifier()

# fit with all_train_embeddings and y_train
clf.fit(X_train_2d, y_train)


# get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(X_test_2d)


# print the classfication report
print(classification_report(y_test, y_pred))